In [7]:
import pandas as pd

# 통합 CSV 불러오기
df = pd.read_csv("통합_전력사용량_데이터.csv")

# 지역명, 연, 월 제거
df_cleaned = df.drop(columns=["지역명", "연", "월"])

# 저장 (선택적)
df_cleaned.to_csv("전처리_데이터.csv", index=False)


In [9]:
from sklearn.preprocessing import MinMaxScaler

# 정규화할 데이터 로드
df_cleaned = pd.read_csv("전처리_데이터.csv")

# 타깃 분리
y = df_cleaned["평균사용량"]
X = df_cleaned.drop(columns=["평균사용량"])

# Min-Max 정규화
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# 타깃 다시 붙이기
df_normalized = pd.concat([X_scaled, y], axis=1)

# 저장 (선택적)
df_normalized.to_csv("정규화_데이터.csv", index=False)


In [15]:
from scipy.stats import ttest_ind
import pandas as pd

# 정규화된 데이터 로드
df = pd.read_csv("정규화_데이터.csv")

# 타깃 컬럼
target_col = "평균사용량"

# 결과 저장용 리스트
ttest_results = []

# 모든 피처에 대해 T-검정 수행
for col in df.columns:
    if col == target_col:
        continue  # 타깃 제외
    
    # 피처의 중간값 기준 상/하 그룹으로 나누고, 각 그룹의 전력사용량 비교
    median_val = df[col].median()
    group_high = df[df[col] >= median_val][target_col]
    group_low = df[df[col] < median_val][target_col]
    
    t_stat, p_value = ttest_ind(group_high, group_low)
    ttest_results.append((col, t_stat, p_value))

# 결과 출력
print("✅ 컬럼별 T-test 결과:")
for col, t_stat, p_val in ttest_results:
    significance = "✔️ 유의함" if p_val < 0.05 else "✖ 유의하지 않음"
    print(f"{col}: t={t_stat:.4f}, p={p_val:.4f} → {significance}")


✅ 컬럼별 T-test 결과:
대상가구수: t=1.6307, p=0.1036 → ✖ 유의하지 않음
평균금액: t=17.6901, p=0.0000 → ✔️ 유의함
평균기온: t=-6.3223, p=0.0000 → ✔️ 유의함
평균최저기온: t=-6.0702, p=0.0000 → ✔️ 유의함
평균최고기온: t=-6.4006, p=0.0000 → ✔️ 유의함
등록공장수: t=-1.4603, p=0.1449 → ✖ 유의하지 않음
총인구수: t=1.6307, p=0.1036 → ✖ 유의하지 않음
세대수: t=1.6307, p=0.1036 → ✖ 유의하지 않음
세대당 인구: t=11.4238, p=0.0000 → ✔️ 유의함
남자 인구수: t=1.6307, p=0.1036 → ✖ 유의하지 않음
여자 인구수: t=1.6307, p=0.1036 → ✖ 유의하지 않음


In [21]:
#전체 컬럼에 대한 모델 학습
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

# 정규화된 데이터 로드
df = pd.read_csv("정규화_데이터.csv")

# 피처/타깃 분리
X = df.drop(columns=["평균사용량"])
y = df["평균사용량"]

# 훈련/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 모델 훈련
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측 및 성능 평가
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("✅ RandomForest 성능:")
print(f"  - R²: {r2:.4f}")


✅ RandomForest 성능:
  - R²: 0.8761


In [19]:
#유의하지 않은 속성을 제외하고 모델 학습
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 정규화된 데이터 불러오기
df = pd.read_csv("정규화_데이터.csv")

# 유의한 변수만 선택
selected_features = ["평균금액", "평균기온", "평균최저기온", "평균최고기온", "세대당 인구"]
X = df[selected_features]
y = df["평균사용량"]

# 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Random Forest 회귀 모델 학습
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측 및 평가
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("✅ 유의한 변수 기반 RandomForest 성능:")
print(f"  - R²: {r2:.4f}")


✅ 유의한 변수 기반 RandomForest 성능:
  - R²: 0.8530
